In [ ]:
# reference mark's deep learning implementation for structured dataset.
# [Part1](https://medium.com/@markryan_69718/deep-learning-on-structured-data-part-1-7f08584b9883)
# [part2](https://medium.com/@markryan_69718/deep-learning-on-structured-data-part-2-5d888b966e9)

In [ ]:
# other references with fastai
# https://www.fast.ai/2018/04/29/categorical-embeddings/
# https://towardsdatascience.com/structured-deep-learning-b8ca4138b848
# https://towardsdatascience.com/deep-learning-structured-data-8d6a278f3088
# https://forums.fast.ai/t/success-with-deep-learning-on-structured-data/19074/4

In [ ]:
# tfreocrd -> serialize example -> high level- > low level -> gpu implement -> set model -> train -> evaluate

In [1]:
import zipfile, os, time, datetime, math, missingno, re, pickle, warnings
import numpy as np
import pandas as pd
from fuzzywuzzy import process
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from itertools import chain

import matplotlib.pyplot as plt
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# what is data serialization?
# https://stackoverflow.com/questions/11817950/what-is-data-serialization

In [3]:
example= tf.train.Example(
        features=tf.train.Features(
            feature={
                "INT_COLOR_CD":tf.train.Feature(int64_list=tf.train.Int64List(value=[1,2,3])),
                }))

In [2]:
def sc_remover(text):
    return re.sub('[^A-Za-z0-9/.()]+', '', text)

In [3]:
def Listelement_str_to_byte(strlist, dim=2):
    if dim==2:
        result = []
        for ii in range(len(strlist)):
            tmp = []
            for jj in range(len(strlist[ii])):
                tmp.append(strlist[ii][jj].encode('utf-8'))
            result.append(tmp)
    elif dim==1:
        result = []
        for ii in range(len(strlist)):
            result.append(strlist[ii].encode('utf-8'))
    else:
        print('pz make custom function')
    return result

In [4]:
# data input
# import 2 xlsx files and merge
filedir = "./Datasets/KMM/20181212/"
filenm = ["Vehicles part 1", "Vehicles part 2"]

for jj in filenm:
    kmm_cust = zipfile.ZipFile(os.path.join(filedir, jj)+".zip")
    kmm_cust.extract(jj+".xlsx", filedir)
    kmm_cust.close()
    
cust1 = pd.read_excel(os.path.join(filedir, filenm[0])+".xlsx")
cust1.rename(columns=dict(cust1.iloc[0,:]), inplace=True)
cust1 = cust1.iloc[1:,:]

cust2 = pd.read_excel(os.path.join(filedir, filenm[1])+".xlsx")
cust2.rename(columns=dict(cust2.iloc[0,:]), inplace=True)
cust2 = cust2.iloc[1:,:]
# sum(cust1.columns != cust2.columns) # if 0 is shown, OK

vehicle = pd.concat([cust1, cust2], axis =0)
vehicle = vehicle.reset_index(drop=True)
# missingno.matrix(vehicle)

In [5]:
# data format convert and cleanup data
# fill in missing values
vehicle['MFGD_DT'] = pd.to_datetime(vehicle['MFGD_DT'])
vehicle['ODOMTR_SALE'] = vehicle['ODOMTR_SALE'].astype(float)
vehicle['MODEL_YR'] = vehicle['MODEL_YR'].astype(str)
vehicle['MFGD_YR'] = vehicle['MFGD_DT'].dt.year.astype(str)
vehicle['MFGD_MN'] = vehicle['MFGD_DT'].dt.month.astype(str)
vehicle['MFGD_DY'] = vehicle['MFGD_DT'].dt.day.astype(str)
vehicle['MFGD_WD'] = vehicle['MFGD_DT'].dt.weekday.astype(str)
vehicle['ODOMTR_SALE'] = vehicle['ODOMTR_SALE'].fillna(0)

vehicle['X_COMPANY_CAR_FLG'] = vehicle.X_COMPANY_CAR_FLG.apply(lambda x: x=='Y')
vehicle['FLEET_FLG'] = vehicle.FLEET_FLG.apply(lambda x: x=='Y')

vehicle['X_TRANSMISSION_DESC'] = vehicle.X_TRANSMISSION_DESC.str.split()
vehicle['X_MODEL_DESCRIPTION'] = vehicle.X_MODEL_DESCRIPTION.str.split()

engine_desc = vehicle.X_ENGINE_DESC.str.split('/')
vehicle['X_ENGINE_DESC1'] = list(zip(*engine_desc))[0]
vehicle['X_ENGINE_DESC2'] = list(zip(*engine_desc))[1]

vehicle = vehicle.drop(['X_ENGINE_DESC',"DEALER_RCPT_DT", "FIRST_SALE_DT", 
                        "MFGD_DT", "X_REGO_DT",'ROW_ID', 'X_ENGINE_NUM','ASSET_NUM', 
                        'X_COMPANY_CAR_FLG', 'FLEET_FLG'], axis = 1)

vehicle['X_TRANSMISSION_DESC'] = pd.Series(Listelement_str_to_byte(vehicle.X_TRANSMISSION_DESC.tolist()))
vehicle['X_MODEL_DESCRIPTION'] = pd.Series(Listelement_str_to_byte(vehicle.X_MODEL_DESCRIPTION.tolist()))

In [13]:
vehicle.head(2)

INT_COLOR_CD   EXT_COLOR_CD MODEL_YR X_SERIES_DESC X_BUILD_PLANT_CD  \
0  STONE BEIGE  REMINGTON RED     2016       SORENTO             KMMG   
1  STONE BEIGE  REMINGTON RED     2016       SORENTO             KMMG   

  X_MODEL_DESC                         X_TRANSMISSION_DESC  \
0           W7  [b'6', b'SPEED', b'2WD', b'/AUTO', b'T/M']   
1           W7  [b'6', b'SPEED', b'2WD', b'/AUTO', b'T/M']   

                                 X_MODEL_DESCRIPTION MFGD_YR MFGD_MN MFGD_DY  \
0  [b'Sorento', b'3.3L', b'EX', b'PACK', b'A/T', ...    2015       7      28   
1  [b'Sorento', b'3.3L', b'EX', b'PACK', b'A/T', ...    2015       7      28   

  MFGD_WD X_ENGINE_DESC1 X_ENGINE_DESC2      y  
0       1            GDI           DOHC  False  
1       1            GDI           DOHC  False

In [6]:
# Define target
# if zero vs one ratio gap is significantly high. need to consider skew data processing and data imbalance.
vehicle['y'] = vehicle.ODOMTR_SALE.apply(lambda x: x>0)

print("zero target values",(vehicle['y']==False).sum())
print("one target values",(vehicle['y']==True).sum())

vehicle = vehicle.drop(['ODOMTR_SALE'], axis=1)

zero target values 105416
one target values 153914


In [8]:
train_else_ratio = 0.4
valid_test_ratio = 0.5

dropout = 0.6
l2_lambda = 7.5
batch_size = 128
epochs = 10
lr = 0.001

In [7]:
# Define feature columns
sequential_columns = ['X_TRANSMISSION_DESC','X_MODEL_DESCRIPTION']
categorical_columns = ["INT_COLOR_CD","EXT_COLOR_CD","MODEL_YR",
                       "X_SERIES_DESC","X_BUILD_PLANT_CD","X_MODEL_DESC",
                       "MFGD_YR","MFGD_MN","MFGD_DY","MFGD_WD","X_ENGINE_DESC1",
                       "X_ENGINE_DESC2"]

In [10]:
# Define test / training sets
train, elsedf = train_test_split(vehicle, test_size = train_else_ratio)

# Split train dateset into train / evaluation
valid, test = train_test_split(elsedf, train_size=valid_test_ratio)

print(train.shape)
print(valid.shape)
print(test.shape)

(155598, 15)
(51866, 15)
(51866, 15)


In [17]:
def make_Example(row):
    example= tf.train.Example(
        features=tf.train.Features(
            feature={
                "INT_COLOR_CD":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['INT_COLOR_CD'].encode('utf-8')])),
                "EXT_COLOR_CD":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['EXT_COLOR_CD'].encode('utf-8')])),
                "MODEL_YR":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['MODEL_YR'].encode('utf-8')])),
                "X_SERIES_DESC":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['X_SERIES_DESC'].encode('utf-8')])),
                "X_BUILD_PLANT_CD":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['X_BUILD_PLANT_CD'].encode('utf-8')])),
                "X_MODEL_DESC":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['X_MODEL_DESC'].encode('utf-8')])),
                "MFGD_YR":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['MFGD_YR'].encode('utf-8')])),
                "MFGD_MN":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['MFGD_MN'].encode('utf-8')])),
                "MFGD_DY":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['MFGD_DY'].encode('utf-8')])),
                "MFGD_WD":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['MFGD_WD'].encode('utf-8')])),
                "X_ENGINE_DESC1":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['X_ENGINE_DESC1'].encode('utf-8')])),
                "X_ENGINE_DESC2":tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['X_ENGINE_DESC2'].encode('utf-8')])),
                'X_TRANSMISSION_DESC':tf.train.Feature(bytes_list=tf.train.BytesList(value= row['X_TRANSMISSION_DESC'])),
                'X_MODEL_DESCRIPTION':tf.train.Feature(bytes_list=tf.train.BytesList(value= row['X_MODEL_DESCRIPTION'])),
                'y':tf.train.Feature(int64_list=tf.train.Int64List(value=[row['y']]))
                }))
    return example

def create_tfrecords_file(tf_filename= './data.tfrecord', input_df=None):
    writer = tf.python_io.TFRecordWriter(tf_filename)
    print("Creating TFRecords file at", tf_filename, "...")
    def mk_tfrecords(row):
        example = make_Example(row)
        content = example.SerializeToString()
        writer.write(content)
        return example
    input_df.apply(mk_tfrecords, axis=1)
    writer.close()
    print("Finish Writing", tf_filename)

In [18]:
create_tfrecords_file("./train.tfrecord", train)
create_tfrecords_file("./valid.tfrecord", valid)
create_tfrecords_file("./test.tfrecord", test)

Creating TFRecords file at ./train.tfrecord ...


Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x0000000035B3DB70>>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'


Finish Writing ./train.tfrecord
Creating TFRecords file at ./valid.tfrecord ...
Finish Writing ./valid.tfrecord
Creating TFRecords file at ./test.tfrecord ...
Finish Writing ./test.tfrecord


In [19]:
read_f = {
    'INT_COLOR_CD': tf.FixedLenFeature([], dtype=tf.string),
    'EXT_COLOR_CD': tf.FixedLenFeature([], dtype=tf.string),
    'MODEL_YR': tf.FixedLenFeature([], dtype=tf.string),
    'X_SERIES_DESC': tf.FixedLenFeature([], dtype=tf.string),
    'X_BUILD_PLANT_CD': tf.FixedLenFeature([], dtype=tf.string),
    'X_MODEL_DESC': tf.FixedLenFeature([], dtype=tf.string),
    'MFGD_YR': tf.FixedLenFeature([], dtype=tf.string),
    'MFGD_MN': tf.FixedLenFeature([], dtype=tf.string),
    'MFGD_DY': tf.FixedLenFeature([], dtype=tf.string),
    'MFGD_WD': tf.FixedLenFeature([], dtype=tf.string),
    'X_ENGINE_DESC1': tf.FixedLenFeature([], dtype=tf.string),
    'X_ENGINE_DESC2': tf.FixedLenFeature([], dtype=tf.string),
    'X_TRANSMISSION_DESC': tf.VarLenFeature(dtype=tf.string),
    'X_MODEL_DESCRIPTION': tf.VarLenFeature(dtype=tf.string),
    'y' : tf.FixedLenFeature([], dtype=tf.int64)
}

def input_fn(filename, predict=False):
    def parser(serialized_exp):
        columns = tf.parse_single_example(serialized=serialized_exp,features=read_f)
        label = columns['y']
        del columns['y']
        features = columns
        return features, label
    dataset= tf.data.TFRecordDataset(filenames=filename)
    if predict == False:
        dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=100,count=-1))
    dataset = dataset.apply(tf.data.experimental.map_and_batch(map_func=parser, batch_size=64))
    dataset = dataset.cache().prefetch(buffer_size = 100) 
    return dataset

In [20]:
def train_input_fn():
    return input_fn(filename='./train.tfrecord')

def eval_input_fn():
    return input_fn(filename='./valid.tfrecord')

def pred_input_fn():
    return input_fn(filename='./test.tfrecord', predict=True)

In [21]:
train_input_fn()

<PrefetchDataset shapes: ({X_MODEL_DESCRIPTION: (?, ?), X_TRANSMISSION_DESC: (?, ?), EXT_COLOR_CD: (?,), INT_COLOR_CD: (?,), MFGD_DY: (?,), MFGD_MN: (?,), MFGD_WD: (?,), MFGD_YR: (?,), MODEL_YR: (?,), X_BUILD_PLANT_CD: (?,), X_ENGINE_DESC1: (?,), X_ENGINE_DESC2: (?,), X_MODEL_DESC: (?,), X_SERIES_DESC: (?,)}, (?,)), types: ({X_MODEL_DESCRIPTION: tf.string, X_TRANSMISSION_DESC: tf.string, EXT_COLOR_CD: tf.string, INT_COLOR_CD: tf.string, MFGD_DY: tf.string, MFGD_MN: tf.string, MFGD_WD: tf.string, MFGD_YR: tf.string, MODEL_YR: tf.string, X_BUILD_PLANT_CD: tf.string, X_ENGINE_DESC1: tf.string, X_ENGINE_DESC2: tf.string, X_MODEL_DESC: tf.string, X_SERIES_DESC: tf.string}, tf.int64)>

In [43]:
# define columns
categorical_feature_columns = [
    tf.feature_column.categorical_column_with_hash_bucket(
        key=j, hash_bucket_size=10**2) for j in categorical_columns]

sequential_feature_columns = [tf.feature_column.categorical_column_with_vocabulary_list(
    key=k, vocabulary_list=list(set(chain.from_iterable(vehicle[k])))) for k in sequential_columns]

feature_columns = [tf.feature_column.embedding_column(l, dimension=100) for l in categorical_feature_columns + sequential_feature_columns]

In [45]:
model = tf.estimator.DNNClassifier(
    n_classes=2,
    dropout=dropout,
    batch_norm=True,
    hidden_units=[128,256],
    model_dir='./',
    feature_columns=feature_columns,
)

model.train(input_fn=train_input_fn, steps=5)
model.evaluate(input_fn=eval_input_fn, steps = 1)

In [ ]:
# 시퀀스 컬럼+ GRU 으로 할 수없을까?

In [ ]:
# use tensorflow
# https://www.tensorflow.org/api_docs/python/tf/contrib/feature_column/sequence_categorical_column_with_hash_bucket
# https://www.tensorflow.org/api_docs/python/tf/contrib/feature_column/sequence_categorical_column_with_identity

tf.contrib.feature_column.sequence_categorical_column_with_vocabulary_list
tf.nn.rnn_cell.GRUCell()
tf.estimator.DNNClassifier()

In [ ]:
predict_result = model.predict(input_fn=test_input_fn)

In [ ]:
est = []
for num, item in enumerate(predict_result):
    if num>=test.shape[0]:
        break
    else:
        est.append(item['class_ids'][0])
est = np.array(est)
act = test.y.values

In [ ]:
colors = sequence_categorical_column_with_vocabulary_list(
    key='colors', vocabulary_list=('R', 'G', 'B', 'Y'),
    num_oov_buckets=2)
colors_embedding = embedding_column(colors, dimension=3)
columns = [colors_embedding]

features = tf.parse_example(..., features=make_parse_example_spec(columns))
input_layer, sequence_length = sequence_input_layer(features, columns)

rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
outputs, state = tf.nn.dynamic_rnn(
    rnn_cell, inputs=input_layer, sequence_length=sequence_length)

In [ ]:
# ROC & AUC
from sklearn.metrics import roc_curve, roc_auc_score

ROC_CURVE = roc_curve(act, est)
FPR = ROC_CURVE[0]
TPR = ROC_CURVE[1]
AUC = roc_auc_score(act, est)

plt.plot(FPR, TPR, '-', label="ROC curve (AUC = %0.2f)" % AUC, color= 'black')
plt.plot([0, 1], [0, 1], 'k--', color = 'lightgrey')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve plot')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# F1 SCORE
from sklearn.metrics import f1_score

f1_score = f1_score(y_true=act, y_pred=est, average='macro')
print("%0.2f" %f1_score)

In [ ]:
# compute the AVERAGE PRECISION SCORE
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(act, est)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
# plot precision-recall curve
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature

precision, recall, _ = precision_recall_curve(act, est)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [8]:
df = vehicle[['X_MODEL_DESC', 'X_TRANSMISSION_DESC']][:1000]
df.columns = ['charcol','listcol']

df['listcol'] = df.listcol.apply(lambda x: [x[i].decode('utf-8') for i in range(len(x))])

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer().fit(df['listcol'])

df1 = pd.get_dummies(df['charcol'])
df1['listcol'] = pd.Series(mlb.transform(df['listcol']).tolist())
df1.columns = ['x1','x2','x3','x4','x5']
df1['y'] = np.random.randint(2,size=1000)

In [9]:
x1 = tf.placeholder(name='x1',shape=[1000,1], dtype=tf.int32)
x2 = tf.placeholder(name='x2',shape=[1000,1], dtype=tf.int32)
x3 = tf.placeholder(name='x3',shape=[1000,1], dtype=tf.int32)
x4 = tf.placeholder(name='x4',shape=[1000,1], dtype=tf.int32)
x5 = tf.placeholder(name='x5',shape=[1000,6], dtype=tf.int32)
y = tf.placeholder(name='y', shape=[1000,1], dtype=tf.int32)

In [10]:
lr1 = tf.layers.dense(units=1024, inputs=tf.concat([x1,x2,x3,x4], axis = 1), activation=tf.nn.relu)

In [15]:
tf.global_variables_initializer()
sess = tf.Session()

sess.run(lr1, feed_dict={
    x1:df1.x1.values.reshape(-1,1), 
    x2:df1.x2.values.reshape(-1,1), 
    x3:df1.x3.values.reshape(-1,1), 
    x4:df1.x4.values.reshape(-1,1)
})

FailedPreconditionError: Attempting to use uninitialized value dense/kernel
	 [[node dense/kernel/read (defined at <ipython-input-10-7e88ed1b6642>:1)  = Identity[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense/kernel)]]

Caused by op 'dense/kernel/read', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_